In [1]:
%%time
%matplotlib inline
import pandas as pd
import numpy as np
import re
import os
import time


raw_path = 'C:\\Finance_team\\raw_data' 
out_path = 'C:\\Finance_team\\Testing_result'
file_list = os.listdir('C:\\Finance_team\\raw_data')

product_id = ''
product_code = ''
date = ''
all_columns = ['시간', '현재가', '체결량', '투자자', '매수1호가수량', '매도1호가수량',
               '투자자매수량', '투자자매도량', '매도or매수']
int_columns = ['현재가', '체결량', '투자자정보', '매수1호가수량', '매도1호가수량',
               '투자자매수량', '투자자매도량', '매도or매수']

def is_target_product_ID(line):
    return line[17:19] == 'A0' and line[425:428] == 'BM3' and line[471:474] == '001'

for i in file_list:
    print(i)
    with open('{0}\\{1}'.format(raw_path, i), 'r', encoding='ISO-8859-1') as f:
        result = []
        lines = f.readlines()
        data_in_array = []

        for line in lines:
            if is_target_product_ID(line):
                product_id = line[53:64]
                product_code = line[35:47]
                date = line[27:35]
                break
        
        for line in lines:
            data_in_array.append(np.array(line.split(',')))
        
        for line in data_in_array:
            
            if (line[0][1:3] < '09'): continue

            if (line[0][1:3] == '15' and line[0][4:6] == '45' and line[0][7:9] >= '01'): break
                  
            if (line[0][22:34] == product_code) or (line[0][38:49] == product_id):
        
                if line[0][17:19] == 'A3':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],np.nan, 
                                    np.nan, np.nan, np.nan, np.nan, line[0][152:153]])
                elif line[0][17:19] == 'B6':
                    result.append([line[0][:17], np.nan, np.nan, np.nan, line[0][57:63], 
                                    line[0][139:145], np.nan, np.nan, np.nan])  
                elif line[0][17:19] == 'G7':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],
                                   np.nan, line[0][169:175], line[0][251:257], 
                                   np.nan, np.nan, line[0][152:153]])
                elif (line[0][17:19] == 'H1' and (line[0][49:53] == '1000')):
                    if (line[0][1:3] == '15' and line[0][4:6] == '35' and line[0][7:9] >= '01'): break
                    result.append([line[0][:17], np.nan, np.nan, line[0][49:53],
                                  np.nan, np.nan, line[0][53:62], line[0][62:71],
                                  np.nan])
            
        result_frame = pd.DataFrame(result, columns = all_columns)
        result_frame.drop(0, inplace=True)
        
result_frame

sise2local.ALL_CMDT.0201
Wall time: 6.8 s


In [2]:
result_frame.head()

,시간,현재가,체결량,투자자,매수1호가수량,매도1호가수량,투자자매수량,투자자매도량,매도or매수
1,[09:00:01-039271],NaN,NaN,NaN,000850,000187,NaN,NaN,NaN
2,[09:00:01-066900],NaN,NaN,NaN,000850,000187,NaN,NaN,NaN
3,[09:00:01-090882],00010745,000001,NaN,000849,000187,NaN,NaN,1
4,[09:00:01-103833],NaN,NaN,NaN,000852,000187,NaN,NaN,NaN
5,[09:00:01-105408],NaN,NaN,NaN,000003,000187,NaN,NaN,NaN


#  H1_Flag Data Frame 생성 

In [4]:
h1 = result_frame.copy()
h1 = h1[(h1['투자자'] == '1000')]
h1.iloc[:, 1:] = h1.iloc[:, 1:].astype('float')

h1 = h1[['시간', '투자자매수량', '투자자매도량']]
# h1['투자자매수량변화(%)'] = round(h1['투자자매수량'].pct_change() * 100, 1)
# h1['투자자매도량변화(%)'] = round(h1['투자자매도량'].pct_change() * 100, 2)
h1['투자자매수량변화'] = h1['투자자매수량'].diff()
h1['투자자매도량변화'] = h1['투자자매도량'].diff()
h1['투자자변화량차이'] = h1['투자자매수량변화'] - h1['투자자매도량변화']

h1['H1_Flag'] = np.nan
h1['H1_Flag'][h1['투자자변화량차이'] > 0] = 1
h1['H1_Flag'][h1['투자자변화량차이'] < 0] = -1
h1['H1_Flag'][h1['투자자변화량차이'] == 0] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
h1.head()

,시간,투자자매수량,투자자매도량,투자자매수량변화,투자자매도량변화,투자자변화량차이,H1_Flag
166,[09:00:36-918856],2734,1239,NaN,NaN,NaN,NaN
227,[09:01:04-218414],3014,1559,280,320,-40,-1.0
267,[09:01:34-112279],3046,1591,32,32,0,0.0
397,[09:02:04-268661],3604,2049,558,458,100,1.0
506,[09:02:34-022089],4294,2612,690,563,127,1.0


# Rest_Flag Data Frame 만들기

In [6]:
# 먼저 Current Price정리 
rest = result_frame.copy()
rest = rest[rest['투자자'] != '1000']
rest = rest[['시간', '현재가', '체결량', '매수1호가수량', '매도1호가수량', '매도or매수']]
rest.iloc[:, 1:] = rest.iloc[:, 1:].astype('float')

rest['현재가'].fillna(method='ffill', inplace=True)
rest['매수1호가수량-매도1호가수량'] = rest['매수1호가수량'] - rest['매도1호가수량']
rest['Rest_Flag'] = np.nan
rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] > 0] = 1
rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] == 0] = 0
rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] < 0] = -1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


# (H1_Flag `+` Rest_Flag) Data Frame 만들기 

In [7]:
df2 = pd.concat([rest, h1])
df2.sort_index(inplace=True)

cols = ['시간', '현재가', '체결량', '매도or매수', '매수1호가수량', '매도1호가수량', 
       '매수1호가수량-매도1호가수량', 'Rest_Flag', '투자자매수량', '투자자매도량',
       '투자자매수량변화', '투자자매도량변화', '투자자변화량차이', 'H1_Flag']

df_test = df2[cols]

df_test['H1_Flag'].fillna(method='ffill', inplace=True)
df_test['Rest_Flag'].fillna(method='ffill', inplace=True)
df_test.loc[0:100]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,시간,현재가,체결량,매도or매수,매수1호가수량,매도1호가수량,매수1호가수량-매도1호가수량,Rest_Flag,투자자매수량,투자자매도량,투자자매수량변화,투자자매도량변화,투자자변화량차이,H1_Flag
1,[09:00:01-039271],NaN,NaN,NaN,850,187,663,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,[09:00:01-066900],NaN,NaN,NaN,850,187,663,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,[09:00:01-090882],10745.0,1,1,849,187,662,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,[09:00:01-103833],10745.0,NaN,NaN,852,187,665,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,[09:00:01-105408],10745.0,NaN,NaN,3,187,-184,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,[09:00:01-168950],10745.0,NaN,NaN,852,187,665,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,[09:00:01-198520],10745.0,NaN,NaN,852,240,612,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,[09:00:01-501859],10745.0,NaN,NaN,852,240,612,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,[09:00:01-931889],10745.0,NaN,NaN,552,240,312,1.0,NaN,NaN,NaN,NaN,NaN,NaN
10,[09:00:02-120073],10745.0,NaN,NaN,552,540,12,1.0,NaN,NaN,NaN,NaN,NaN,NaN


# 전략 검증 위해 Test Frame 생성 as 'a'

In [8]:
a = df_test[['시간', '현재가', '매수1호가수량', '매도1호가수량', 'H1_Flag', 'Rest_Flag']]

In [9]:
a.loc[350:400]

,시간,현재가,매수1호가수량,매도1호가수량,H1_Flag,Rest_Flag
350,[09:01:55-793040],10746.0,159,845,0.0,-1.0
351,[09:01:55-799100],10746.0,156,845,0.0,-1.0
352,[09:01:55-800526],10746.0,156,870,0.0,-1.0
353,[09:01:55-803099],10746.0,1248,870,0.0,1.0
354,[09:01:55-806673],10746.0,1223,870,0.0,1.0
355,[09:01:56-252896],10746.0,1223,880,0.0,1.0
356,[09:01:56-433787],10746.0,1232,880,0.0,1.0
357,[09:01:56-434856],10746.0,1246,880,0.0,1.0
358,[09:01:56-695485],10746.0,1232,880,0.0,1.0
359,[09:01:56-696706],10746.0,1223,880,0.0,1.0


# H1_Flag 및 Rest_Flag 의 0 값을 np.nan값으로 변환

In [14]:
a['H1_Flag'][a['H1_Flag'] == 0] = np.nan
a['Rest_Flag'][a['Rest_Flag'] == 0] = np.nan

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C

# H1_Flag 및 Rest_Flag의 np.nan 값을 fillna의 method='ffill'을 활용해서 이전 값 참조

In [15]:
a['H1_Flag'].fillna(method='ffill', inplace=True)
a['Rest_Flag'].fillna(method='ffill', inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [19]:
a.loc[200:450]

,시간,현재가,매수1호가수량,매도1호가수량,H1_Flag,Rest_Flag
200,[09:00:47-169815],10747.0,655,174,NaN,1.0
201,[09:00:47-169971],10747.0,655,174,NaN,1.0
202,[09:00:47-171160],10747.0,655,174,NaN,1.0
203,[09:00:47-172368],10747.0,655,174,NaN,1.0
204,[09:00:48-816921],10747.0,655,164,NaN,1.0
205,[09:00:48-820122],10747.0,655,161,NaN,1.0
206,[09:00:49-410653],10747.0,655,167,NaN,1.0
207,[09:00:50-888896],10747.0,636,167,NaN,1.0
208,[09:00:51-137351],10747.0,636,117,NaN,1.0
209,[09:00:51-138716],10747.0,655,117,NaN,1.0


In [120]:
renew_index = 0
# print(renew_index, '#0')

while True:
    for index, row in a.loc[renew_index:].iterrows():

        if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1: 
            renew_index = index
            print(row['H1_Flag'], row['Rest_Flag'])
#             print(renew_index,'#1')
            
            for index, row in a.loc[renew_index:].iterrows():

                if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1:
                    renew_index = index
                    print(row['H1_Flag'], row['Rest_Flag'])

#                     print(renew_index, '#2')
                    break
#             print(renew_index, '#3')
            print('Next Search:', renew_index)
            break
    if index == len(a.loc[:]):
        break

        
# 여기서, index와 row 를 활용해서 시그널을 제대로 찾아야 함.
# 집중해라 여기만 통과하면 끝이다.

# omg...

-1.0 1.0
-1.0 -1.0
Next Search: 231
-1.0 1.0
-1.0 -1.0
Next Search: 344
-1.0 1.0
-1.0 -1.0
Next Search: 375
-1.0 1.0
-1.0 -1.0
Next Search: 551
-1.0 1.0
-1.0 -1.0
Next Search: 590
-1.0 1.0
-1.0 -1.0
Next Search: 1135
-1.0 1.0
-1.0 -1.0
Next Search: 1161
-1.0 1.0
-1.0 -1.0
Next Search: 1193
-1.0 1.0
-1.0 -1.0
Next Search: 1206
-1.0 1.0
-1.0 -1.0
Next Search: 1218
-1.0 1.0
-1.0 -1.0
Next Search: 1357
-1.0 1.0
-1.0 -1.0
Next Search: 1459
-1.0 1.0
-1.0 -1.0
Next Search: 1470
-1.0 1.0
-1.0 -1.0
Next Search: 1473
-1.0 1.0
-1.0 -1.0
Next Search: 1478
-1.0 1.0
-1.0 -1.0
Next Search: 1480
-1.0 1.0
-1.0 -1.0
Next Search: 1486
-1.0 1.0
-1.0 -1.0
Next Search: 1488
-1.0 1.0
-1.0 -1.0
Next Search: 1494
-1.0 1.0
-1.0 -1.0
Next Search: 1503
-1.0 1.0
-1.0 -1.0
Next Search: 1512
-1.0 1.0
-1.0 -1.0
Next Search: 1515
-1.0 1.0
-1.0 -1.0
Next Search: 1520
-1.0 1.0
-1.0 -1.0
Next Search: 1538
-1.0 1.0
-1.0 -1.0
Next Search: 1660
-1.0 1.0
-1.0 -1.0
Next Search: 2002
-1.0 1.0
-1.0 -1.0
Next Search: 2037
-1.0 1

In [30]:
a.loc[225:250]

,시간,현재가,매수1호가수량,매도1호가수량,H1_Flag,Rest_Flag
225,[09:01:03-669672],10747.0,591,192,NaN,1.0
226,[09:01:03-779553],10747.0,591,242,NaN,1.0
227,[09:01:04-218414],NaN,NaN,NaN,-1.0,1.0
228,[09:01:04-334976],10747.0,591,267,-1.0,1.0
229,[09:01:04-604190],10747.0,591,267,-1.0,1.0
230,[09:01:04-684682],10747.0,591,567,-1.0,1.0
231,[09:01:04-686354],10747.0,513,567,-1.0,-1.0
232,[09:01:04-875886],10747.0,513,582,-1.0,-1.0
233,[09:01:06-515678],10747.0,513,582,-1.0,-1.0
234,[09:01:08-442669],10747.0,513,583,-1.0,-1.0


In [116]:
len(a.loc[:])

15973